<a href="https://colab.research.google.com/github/Stanfording/Simple-AE/blob/main/simple_Unet_No_skip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Goal:   Just get a small GAN running



Get the preprocessed celebHD data from google drive

In [ ]:
#Download the dataset
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1KqBRLsB0CJuQGycvaPINwaPgcGDUsAxN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1KqBRLsB0CJuQGycvaPINwaPgcGDUsAxN" -O "proCeleba.zip" && rm -rf /tmp/cookies.txt

#unzip the dataset
!unzip "/content/proCeleba.zip"

#remove unnecessary files
!rm -rf /content/__MACOSX

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Install oneflow framework

In [ ]:
#!python3 -m pip install -f https://release.oneflow.info oneflow==0.7.0+cu102

In [ ]:
#import oneflow as t

### Import libraries

In [ ]:
!pip install torchmetrics

In [ ]:
from tqdm import tqdm

from torch.autograd import Variable, grad
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import os
import torch as t
import torch.nn as nn
from torchvision import datasets, transforms, utils
from torch.utils.data import Dataset, DataLoader
from skimage import io
import copy
# from torchmetrics.classification import BinaryAccuracy

### Set initial hyperparameters

In [ ]:

batch_size = 32

x_resolution = 16

y_resolution = 128 * 2

img_fold_dir_16_reso = f"/content/proCeleba/{x_resolution}"

img_fold_dir_64_reso = f"/content/proCeleba/{y_resolution}"

iteration = 200

critic = 20          #Train generator 5 times per epoch

eval_size = 25

laten_space = 100

updatD_step = 5

device = t.device('cuda' if t.cuda.is_available() else 'cpu')

log_folder = "log"
!mkdir "log"
!mkdir "log/checkpoint"
!mkdir "log/sample"

### Show me which gpu I am using.

In [ ]:
!nvidia-smi -L

### Pre loading the data

In [ ]:
# Define a data class for load unclassfied data.
class Get_No_Classes_Img_Dataset(Dataset):

    def __init__(self, folder_dir, transform = None, addNoise = False):
        self.folder_dir = os.path.join(folder_dir)
        self.transform = transform
        self.image_list = os.listdir(self.folder_dir)
        self.addNoise = addNoise


    def __len__(self):
        return len(os.listdir(self.folder_dir))

    def __getitem__(self, index):

        image_name = self.image_list[index]

        image_dir = os.path.join(self.folder_dir, image_name)

        image = io.imread(image_dir)

        if (self.transform != None):
            image = self.transform(image)
            if self.addNoise == True:
              noise = t.randn_like(image)
              image += noise


        return image

# transform_X = transforms.ToTensor()
transform_Y = transforms.ToTensor() #From Batch * Highth * Width * Channel to Batch * Channel * Highth * Width
                                  #Which is what pytorch CNN can work with.

# X_dataset = Get_No_Classes_Img_Dataset(img_fold_dir_16_reso, transform = transform_X, addNoise = True)

Y_dataset = Get_No_Classes_Img_Dataset(img_fold_dir_64_reso, transform = transform_Y)
                                                            # datasets[0].shape = (16,16,3)
                                                            # len(datasets) = 28000
total_data_len = len(Y_dataset)

# X_dataloader = DataLoader(X_dataset, batch_size = batch_size) #loader is renewed every epoch
Y_dataloader = DataLoader(Y_dataset, batch_size = batch_size) #loader is renewed every epoch

# X_dataloader = list(iter(X_dataloader))
Y_dataloader = list(iter(Y_dataloader))




```
loader = iter(loader)
print(next(loader).shape)
```
will output


```
torch.Size([batch_size, 3, resolution, resolution])
```

So data loading is ready.

What's left is keep using

```
next(loader)
```
to access each batch of data


### Visualize a picture

In [ ]:
from IPython.display import Image, display




# Viewing one data sample function:
def showOneImge(img, i, shouldSave):

    img = img.squeeze()

    img = transforms.ToPILImage()(img)
    #img = transforms.GaussianBlur(15, 3)(img)

    plt.figure(figsize = (15,15), dpi = 30)
    plt.axis('off')

    if shouldSave:
      saveDir = f'{log_folder}/sample/{str(i).zfill(6)}.png'
      plt.imshow(img)
      plt.savefig(saveDir, bbox_inches='tight', pad_inches = 0)
      img = Image(saveDir)
      display(img)
    else:
      deleteDir = f"{log_folder}/sample/Delete.png"
      plt.imshow(img)
      plt.savefig(deleteDir, bbox_inches='tight', pad_inches = 0)
      img = Image(deleteDir)
      display(img)
      !rm "/content/log/sample/Delete.png"
    return




def showMoreImages(img, num):

  subplot_x = int(num ** (1/2))
  subplot_y = num // subplot_x
  plt.figure(figsize = (2,2))
  for i in range(len(img)):
      aimg = transforms.ToPILImage()(img[i])
      plt.subplot(subplot_x, subplot_y, i+1)
      plt.imshow(aimg)
      plt.axis('off')


Visulize Data

In [ ]:
''' Testing showOneImage'''

# oneSample_X = X_dataloader[0][0]
oneSample_Y = Y_dataloader[0][0]
print(oneSample_Y.shape)

# showOneImge(oneSample_X, 9999, False)
showOneImge(oneSample_Y, 9999, False)

-------------------------------------------------

### Now Designing the simple network

In [ ]:
"""
Define the GAN Network
"""
import math

class MLP_GAN(nn.Module):

  def __init__(self):
    super().__init__()

    self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 64, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 3, (1, 1), padding = "same"),
            nn.Sigmoid()
            )

  def forward(self, theInput):
    output = self.layer1(theInput)
    return output



class G(nn.Module):

    def __init__(self):
        super().__init__()

        self.upsample = nn.UpsamplingBilinear2d(y_resolution)

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.InstanceNorm2d(64),
            nn.Conv2d(64, 256, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(),
            nn.InstanceNorm2d(256),
            nn.Conv2d(256, 256, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(),
            nn.InstanceNorm2d(256),
            nn.Conv2d(256, 256, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(),
            nn.InstanceNorm2d(256),
            nn.Conv2d(256, 256, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.InstanceNorm2d(256),
            nn.Conv2d(256, 32, (3, 3), padding = "same"),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(),
            nn.InstanceNorm2d(32),
            nn.Conv2d(32, 3, (3, 3), padding = "same"),
            nn.Sigmoid()
        )



    def forward(self, theInput):

        x = self.upsample(theInput)

        x = self.model(x)


        return x

class D(nn.Module):

    def __init__(self):
        super().__init__()
        self.netWork = nn.Sequential(
            nn.Conv2d(3, 64, (3, 3), padding = "same"),
            nn.LeakyReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(64, 256, (3, 3), padding = "same"),
            nn.LeakyReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(256, 256, (3, 3), padding = "same"),
            nn.LeakyReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(256, 256, (3, 3), padding = "same"),
            nn.LeakyReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(256, 256, (3, 3), padding = "same"),
            nn.LeakyReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(256, 128, (3, 3), padding = "same"),
            nn.LeakyReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(128, 8, (3, 3), stride = (3, 3)))

        self.output = nn.Sequential(
            nn.BatchNorm2d(8),
            nn.LeakyReLU(),
            nn.Conv2d(8, 1, (3, 3), stride = (3, 3)),
            nn.AvgPool2d(7,7),
            nn.Flatten())




    def forward(self, theInput):
        latent = self.netWork(theInput)
        output = self.output(latent)



        return output, latent

"""
Define A simple Auto-Encoder network
"""

#Modified the design of https://colab.research.google.com/drive/1sjy9odlSSy0RBVgMTgP7s99NXsqglsUL?usp=sharing#scrollTo=KOYPSxPf_LL7

class Block(nn.Module):
    def __init__(self, in_ch, out_ch, up=False):
        super().__init__()
        if up:
            self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
            self.transform = nn.ConvTranspose2d(out_ch, out_ch, 4, 2, 1)
        else:
            self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
            self.transform = nn.Conv2d(out_ch, out_ch, 4, 2, 1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.bnorm1 = nn.BatchNorm2d(out_ch)
        self.bnorm2 = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU()

    def forward(self, x):
        # First Conv
        h = self.bnorm1(self.relu(self.conv1(x)))
        # Add time channel
        # Second Conv
        h = self.bnorm2(self.relu(self.conv2(h)))
        # Down or Upsample
        return self.transform(h)

class Layer(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()

        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.bnorm1 = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU()

    def forward(self, x):
        return self.bnorm1(self.relu(self.conv1(x)))



class SimpleUnet(nn.Module):
    """
    A simplified variant of the Unet architecture.
    """
    def __init__(self):
        super().__init__()
        image_channels = 3
        down_channels = (32, 64, 128, 256, 512, 1024)
        up_channels = (1024, 512, 256, 128, 64,32)
        out_dim = 1

        # Initial projection
        self.conv0 = nn.Conv2d(image_channels, down_channels[0], 3, padding=1)

        # Downsample
        self.downs = nn.ModuleList([Block(down_channels[i], down_channels[i+1]) \
                    for i in range(len(down_channels)-1)])
        # Upsample
        self.ups = nn.ModuleList([Block(up_channels[i], up_channels[i+1], up=True) \
                    for i in range(len(up_channels)-1)])

        self.output = nn.Sequential(
            Layer(up_channels[-1], up_channels[-1]),
            Layer(up_channels[-1], up_channels[-1]),
            Layer(up_channels[-1], up_channels[-1]),
            nn.Conv2d(up_channels[-1], 3, out_dim),
            nn.Sigmoid())

    def forward(self, x):
        # Embedd time
        # Initial conv
        x = self.conv0(x)
        # Unet
        for down in self.downs:
            x = down(x)
        for up in self.ups:
            # Add residual x as additional channels
            x = up(x)


        return self.output(x)





### Testing the network

In [ ]:
"""

Testing the GAN network:

"""

oneSample_X = X_dataloader[0]
oneSample_Y = Y_dataloader[0]

#img into G to test shape

a = G()(oneSample_X)

showOneImge(a[0], 0, shouldSave = False)

b, latent = D()(oneSample_Y)

c = MLP_GAN()

output = c(oneSample_Y)


print(b[0])
print(latent.shape)

showOneImge(output[0], 0, shouldSave = False)

### Gradient Penalty from wGAN, will test later.

In [ ]:
# def compute_gradient_penalty(D, real_samples, fake_samples, current_batch_size):
#     """Calculates the gradient penalty loss for WGAN GP"""
#     # Random weight term for interpolation between real and fake samples
#     alpha = t.randn((current_batch_size, 1, 1, 1)).to(device)
#     # Get random interpolation between real and fake samples
#     interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)


#     d_interpolates = D(interpolates)


#     grad_x_hat = grad(
#             outputs=d_interpolates.sum(), inputs=interpolates, create_graph=True)[0]
#     grad_penalty = ((grad_x_hat.view(grad_x_hat.size(0), -1)
#                       .norm(2, dim=1) - 1)**2).mean()
#     grad_penalty = 10 * grad_penalty

#     return grad_penalty

### Initializing the generator, discriminator, optimizer, labels, and loss.

In [ ]:
generator = G().to(device)
discriminator = D().to(device)
modifying_layer = MLP_GAN().to(device)

G_optimizer = t.optim.Adam(generator.parameters(), lr = 0.0001)
D_optimizer = t.optim.Adam(discriminator.parameters(), lr = 0.0001)
Modify_optimizer = t.optim.Adam(modifying_layer.parameters(), lr = 0.0001)

Unet_no_skip = SimpleUnet().cuda()
U_optim = t.optim.Adam(Unet_no_skip.parameters(), lr = 0.001)


# acc = BinaryAccuracy().to(device)

bceLoss = nn.BCEWithLogitsLoss()
mseLoss = nn.MSELoss()
L1_Loss = nn.L1Loss()

should_train_G = True #True = train d, False = train g

### Train decider

In [ ]:
# def what_to_train(fake_score, real_score, should_train_G):
#   last_should_train_G = should_train_G


#   if (fake_score < real_score * score_dif):
#     return True

#   if last_should_train_G:
#     if (fake_score > real_score * (1 - score_dif)):
#       return False
#     else:
#       return True

#   return False







### Start training

Try Simple AE Unet without skip connection

In [ ]:
best_mse = 99999
for epoch in range(iteration):
    Unet_no_skip.train()
    pbar = tqdm(enumerate(Y_dataloader))

    mse_average = 0

    for i, X_data in pbar:


        epochNum = str(epoch)

        #Training the D
        #data
        real = Y_dataloader[i].cuda()

        #current_batch size (the last batch is different than others)
        current_batch_size, c, h, w = real.shape

        #Train G
        # canvas = t.ones((real.shape)).cuda()
        pre = Unet_no_skip(real) #+ canvas

        loss = mseLoss(pre, real).mean()
        Unet_no_skip.zero_grad()
        loss.backward()
        U_optim.step()

        mse_average += loss.item()

        mse = "Epoch: " + epochNum

        pbar.set_description(mse)

        pbar.set_postfix(MSELoss = loss.item())

    mse_average /= len(Y_dataloader)

    if epoch % 1 == 0:

      showOneImge(real[0], 99999, False)
      showOneImge(pre[0], 99999, False)
      showOneImge(real[1], 99999, False)
      showOneImge(pre[1], 99999, False)
      showOneImge(real[2], 99999, False)
      showOneImge(pre[2], 99999, False)
      print("epoch = ", epoch + 1)
      print('Epoch [{}/{}], Step [{}/{}], avg_mse_loss: {:.4f}'.format(epoch, iteration, i+1, total_data_len // batch_size + 1, mse_average))


    if mse_average < best_mse:
      t.save(Unet_no_skip.state_dict(), f'{log_folder}/checkpoint/{str(epoch + 1).zfill(6)}_best_Unet.pth')
      best_mse = mse_average


In [ ]:
cp /content/log/checkpoint /content/drive/MyDrive/Event_TO_PC

Testing GAN on top of regular ae

In [ ]:

boost_loop = 2

d_loop = 3

trainG = True

for epoch in range(iteration):

    pbar = tqdm(enumerate(X_dataloader))

    # real_score = t.Tensor(2)

    # fake_score = t.Tensor(2)

    # g_fake_loss = t.Tensor([2])

    # g_total_loss = 0

    d_step = 0
    fake_score = t.tensor(float('inf'))
    mse_average = 0
    real_score_mean = 0
    fake_score_mean = 0
    accuracy_mean = 0
    mse_loss = t.tensor(float('inf'))
    real_score = t.tensor(float('inf'))
    d_real_loss = t.tensor(float('inf'))
    d_fake_loss = t.tensor(float('inf'))
    total_loss = t.tensor(float('inf'))
    accuracy = t.tensor(float('inf'))

    for i, X_data in pbar:


        epochNum = str(epoch)

        #Training the D
        #data
        real = Y_dataloader[i].cuda()
        X_data = X_data.cuda()

        batch_size = X_data.shape[0]

        label_real = t.ones((batch_size, 1)).cuda()
        label_fake = t.zeros((batch_size, 1)).cuda()

        #current_batch size (the last batch is different than others)
        current_batch_size, c, h, w = real.shape


        #Train G


        # mse_average += mse_loss
        fake = generator(X_data)

        mse_loss_image = mseLoss(fake, real)
        fake_score, latent_fake = discriminator(fake)
        real_score, latent_real = discriminator(fake)
        mse_loss_latent = mseLoss(latent_fake, latent_real)
        g_fake_loss = mse_loss_latent  + mse_loss_image  #bceLoss(fake_score, label_real).mean()

        discriminator.zero_grad()
        generator.zero_grad()
        g_fake_loss.backward()
        G_optimizer.step()

        mse_average += mse_loss_image.item()

        #Train D
        real_score, latent_real = discriminator(real)
        fake_score, latent_fake = discriminator(fake.detach().clone())

        accuracy = acc(fake_score, label_fake)
        accuracy_mean += accuracy.item()

        # d_fake_loss = t.mean(fake_score)


        d_fake_loss = bceLoss(fake_score, label_fake).mean()
        fake_score_mean += fake_score.mean().item()
        d_real_loss = bceLoss(real_score, label_real).mean()
        real_score_mean += real_score.mean().item()

        # gradient_penalty = compute_gradient_penalty(discriminator, real, fake, current_batch_size)

        total_loss = d_fake_loss + d_real_loss #+ gradient_penalty
        # total_loss = -real_score.mean() + fake_score.mean()#d_fake_loss + d_real_loss

        generator.zero_grad()
        discriminator.zero_grad()
        total_loss.backward()
        D_optimizer.step()







        mse = "Epoch: " + epochNum

        pbar.set_description(mse)

        pbar.set_postfix(real_score = real_score.mean().item(), fake_score = fake_score.mean().item(), mse_loss = mse_loss_image.item(), accuracy = accuracy.item())

    mse_average /= len(X_dataloader)
    real_score_mean /= len(X_dataloader)
    fake_score_mean /= len(X_dataloader)
    accuracy_mean /= len(X_dataloader)

    if epoch % 1 == 0:

      showOneImge(real[0], 99999, False)
      showOneImge(fake[0], 99999, False)
      showOneImge(real[1], 99999, False)
      showOneImge(fake[1], 99999, False)
      showOneImge(real[2], 99999, False)
      showOneImge(fake[2], 99999, False)
      print("epoch = ", epoch + 1)
      print('Epoch [{}/{}], Step [{}/{}], real_mean: {:.2f}, fake_mean: {:.2f}, avg_mse: {:.2f}, accuracy_mean: {:.2f}'.format(epoch, iteration, i+1, total_data_len // batch_size + 1, real_score_mean, fake_score_mean, mse_average, accuracy_mean))


    if i % 50 == 0:
      t.save(generator.state_dict(), f'{log_folder}/checkpoint/{str(i + 1).zfill(6)}_g.model')
      t.save(discriminator.state_dict(), f'{log_folder}/checkpoint/{str(i + 1).zfill(6)}_d.model')


### Show Result

In [ ]:


#Generated
generator.eval()

with t.no_grad():
  input_noise = t.normal(0, 1, size = (eval_size, 1, 1, laten_space)).to(device)
  generated = generator(input_noise, eval_size)
  showMoreImages(generated, eval_size)

#Real
datasets_batched = DataLoader(dataset, batch_size = eval_size)
loader = iter(datasets_batched)
real = next(loader)
showMoreImages(real, eval_size)

### remove log file when necessarry

In [ ]:
# !rm -rf /content/log/checkpoint

# !mkdir /content/log/checkpoint

# !rm -rf /content/log/sample
# !mkdir /content/log/sample

# from google.colab import files
# files.download('/content/log')